In [ ]:
from huggingface_hub import notebook_login

# Paste your token when prompted
notebook_login()


In [ ]:
import os

# Replace YOUR_TOKEN with your actual Hugging Face token
os.environ['HF_HOME'] = '/content/cache'
os.environ['TRANSFORMERS_CACHE'] = '/content/cache'
os.environ['HF_DATASETS_CACHE'] = '/content/cache'
os.environ['HF_METRICS_CACHE'] = '/content/cache'
os.environ['HF_HUB_CACHE'] = '/content/cache'
os.environ['HF_TOKEN'] = 'YOUR_TOKEN'


In [ ]:
!pip install torch transformers datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 4.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
import torch
from torch import nn, optim
from transformers import GPT2Tokenizer
from datasets import load_dataset
from torch.utils.data import DataLoader


/usr/local/lib/python3.10/dist-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained('distilgpt2')
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token  # Set pad token if it's not set

dataset = load_dataset("Amod/mental_health_counseling_conversations")

def encode_conversations(examples):
    inputs = tokenizer(examples['Context'], padding="max_length", truncation=True, max_length=128)  # Reduced max_length
    outputs = tokenizer(examples['Response'], padding="max_length", truncation=True, max_length=128)  # Reduced max_length
    return {'input_ids': inputs['input_ids'], 'labels': outputs['input_ids']}

dataset = dataset.map(encode_conversations, batched=True, remove_columns=dataset['train'].column_names)


/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


In [ ]:
class SimpleSeq2Seq(nn.Module):
    def __init__(self, vocab_size, embedding_dim, hidden_dim):
        super(SimpleSeq2Seq, self).__init__()
        self.embedding = nn.Embedding(vocab_size, embedding_dim)
        self.encoder = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.decoder = nn.GRU(embedding_dim, hidden_dim, batch_first=True)
        self.fc = nn.Linear(hidden_dim, vocab_size)

    def forward(self, input_ids, labels):
        embedded = self.embedding(input_ids)
        _, hidden = self.encoder(embedded)  # Only pass the hidden state to the decoder

        decoder_input = self.embedding(labels)
        output, _ = self.decoder(decoder_input, hidden)  # Initial hidden state from encoder
        prediction = self.fc(output)
        return prediction


In [ ]:
vocab_size = tokenizer.vocab_size
embedding_dim = 128  # Reduced embedding dimension
hidden_dim = 256  # Reduced hidden dimension

model = SimpleSeq2Seq(vocab_size, embedding_dim, hidden_dim)
optimizer = optim.Adam(model.parameters(), lr=0.001)
criterion = nn.CrossEntropyLoss(ignore_index=tokenizer.pad_token_id)


In [ ]:
def collate_fn(batch):
    input_ids = torch.tensor([item['input_ids'] for item in batch])
    labels = torch.tensor([item['labels'] for item in batch])
    return {'input_ids': input_ids, 'labels': labels}

train_loader = DataLoader(dataset['train'], batch_size=8, shuffle=True, collate_fn=collate_fn)


In [ ]:
!pip install tqdm


In [ ]:
from tqdm import tqdm  # Import tqdm for the progress bar

def train(model, loader, optimizer, criterion, epochs):
    model.train()
    for epoch in range(epochs):
        total_loss = 0
        progress_bar = tqdm(loader, desc=f'Epoch {epoch+1}', leave=True)  # Initialize the progress bar
        for batch in progress_bar:
            optimizer.zero_grad()
            output = model(batch['input_ids'], batch['labels'])
            loss = criterion(output.transpose(1, 2), batch['labels'])
            loss.backward()
            optimizer.step()
            total_loss += loss.item()
            progress_bar.set_postfix(loss=loss.item())  # Update the progress bar with current loss
        print(f"Epoch {epoch+1}, Average Loss: {total_loss / len(loader)}")
train_loader


In [ ]:
train(model, train_loader, optimizer, criterion, epochs=3)

Epoch 1: 100%|██████████| 439/439 [37:13<00:00,  5.09s/it, loss=0.284]


Epoch 1, Average Loss: 1.963717015430433


Epoch 2: 100%|██████████| 439/439 [36:49<00:00,  5.03s/it, loss=0.0481]


Epoch 2, Average Loss: 0.2166008481454605


Epoch 3: 100%|██████████| 439/439 [36:44<00:00,  5.02s/it, loss=0.0189]

Epoch 3, Average Loss: 0.05010217103517375
